In [ ]:
import os.path as osp

# computation
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# data pipeline
import imageio
#from imgaug import augmenters as iaa
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms as transforms

# utils
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler

# Dataset

In [ ]:
# Some constants
ROOT = '/kaggle/input/cassava-leaf-disease-classification'
TRAIN_DIR = f'{ROOT}/train_images/'
TRAIN_CSV = f'{ROOT}/train.csv'
TEST_DIR = f'{ROOT}/test_images/'
TEST_CSV = f'{ROOT}/sample_submission.csv'

In [ ]:
class CassavaDataset(Dataset):
    def __init__(self, split, transform=None):
        assert split in ('train', 'val', 'test')
        self.split = split
        self.transform = transform
        if split in ('train', 'val'):
            csv = pd.read_csv(TRAIN_CSV)
            
            #csv = csv[:10000]  # 데이터 개수 조절
            
            self.df = train_test_split(
                csv, test_size=0.1, random_state=0,stratify=csv[['label']]
            )[0 if split == 'train' else 1].reset_index()
        else:
            self.df = pd.read_csv(TEST_CSV)
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, i: int):
        base_dir = TRAIN_DIR if self.split in ('train', 'val') else TEST_DIR
        x = imageio.imread(osp.join(base_dir, self.df['image_id'][i]))
        y = self.df['label'][i] if self.split in ('train', 'val') else -1
        if self.transform:
            x = self.transform(x)
        return (x, y)

# Implement your data augmentation pipelines

In [ ]:
# TODO: You'll need some heavy augmentations to get a high score. 
#
# See 
#     https://github.com/aleju/imgaug 
#     https://imageio.readthedocs.io/en/stable/
#
# for detailed `imgaug` references.
#
# NOTE: If you choose to normalize the training images, you should normalize the test images as well.
#
INPUT_SIZE = 224
RESIZE = 500
TRANSFORMS = {
    'train': transforms.Compose([
        
#         iaa.Sequential([
#             iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
#             iaa.Fliplr(0.5),
#         ]).augment_image,  
        transforms.ToPILImage(),
        transforms.Resize((RESIZE,RESIZE)),
        transforms.RandomCrop((INPUT_SIZE,INPUT_SIZE)),
        #transforms.RandomResizedCrop(224),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomHorizontalFlip(p=0.2),        
        transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'val': transforms.Compose([
#         iaa.Sequential([
#             iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
#         ]).augment_image,
        transforms.ToPILImage(),
        transforms.Resize((RESIZE,RESIZE)),
        transforms.RandomCrop((INPUT_SIZE,INPUT_SIZE)),
        #transforms.Resize((INPUT_SIZE,INPUT_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'test': transforms.Compose([
#         iaa.Sequential([
#             iaa.Resize((INPUT_SIZE, INPUT_SIZE)),
#         ]).augment_image,
        transforms.ToPILImage(),
        transforms.Resize((RESIZE,RESIZE)),
        transforms.RandomCrop((INPUT_SIZE,INPUT_SIZE)),
        #transforms.Resize((INPUT_SIZE,INPUT_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}

In [ ]:
batch_size = 128

train_dataset = CassavaDataset('train', transform=TRANSFORMS['train'])

val_dataset = CassavaDataset('val', transform=TRANSFORMS['val'])

test_dataset = CassavaDataset('test', transform=TRANSFORMS['test'])

In [ ]:
# y_train = []
# for i in range(len(train_dataset)):
#     y_train.append(train_dataset[i][1])
# y_train = np.array(y_train)
y_train=np.array(train_dataset.df['label'])

class_sample_count = np.array([len(np.where(y_train==t)[0]) for t in np.unique(y_train)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in y_train])

samples_weight = torch.from_numpy(samples_weight)

train_sampler = WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight))
          

In [ ]:
y_train.shape

In [ ]:
# hist = np.array([0, 0, 0, 0, 0])
# for x,y in train_loader:
#     y = np.array(y)
#     for i in range(len(y)):
#         #print(y[i])
#         hist[y[i]] += 1
# print(hist)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
batch_size = 64

train_dataset = CassavaDataset('train', transform=TRANSFORMS['train'])
train_loader = DataLoader(train_dataset, batch_size=batch_size)

val_dataset = CassavaDataset('val', transform=TRANSFORMS['val'])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = CassavaDataset('test', transform=TRANSFORMS['test'])
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
print("training data:",len(train_loader.dataset))
print("validation data:",len(val_loader.dataset))
print("test data:",len(test_loader.dataset))

# Build your Cassava leaf classifier

In [ ]:
# import torchvision.models as models
# class Cassavaclassfier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.resnet = models.resnet50()
#         self.resnet.fc = nn.Linear(sle.fresnet.fc.in_features, 5)
    
#     def forward(self, x):
#         return self.resnet(x)
    
# model = Cassavaclassfier().cuda()

In [ ]:

# TODO: Implement your classifier.
class CassavaClassifier(nn.Module):
    def __init__(self):

        super().__init__()
        CH_LAST = 128
        NUM_CLASSES = 5
        # TODO: Change this part to improve your score
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, CH_LAST, kernel_size=3, padding=1)
        # NOTE: input channel should match CH_LAST * (IMAGE_INPUT_SIZE / 2 ** NUM_DOWNSAMPLE) ** 2 
        self.fc = nn.Linear(CH_LAST *  (INPUT_SIZE // 2 ** 3) ** 2, NUM_CLASSES)
        self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1)
        
    def forward(self, x):
        h = self.pool(F.relu(self.conv1(x)))
        h = self.pool(F.relu(self.conv2(h)))
        h = self.pool(F.relu(self.conv3(h)))
        return self.fc(h.flatten(1))

model = CassavaClassifier().cuda()
#model

In [ ]:
# import torchvision.models as models
# class Cassavaclassfier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.mobilenet = models.mobilenet_v2()
#         self.mobilenet.fc = nn.Linear(sle.fresnet.fc.in_features, 5)
    
#     def forward(self, x):
#         return self.mobilenet(x)
    
# model = Cassavaclassfier().cuda()

# #import torchvision.models as models
# #model = models.mobilenet_v2().cuda()

In [ ]:
# # TODO: Implement your classifier.
# class CassavaClassifier(nn.Module):
#     def __init__(self):

#         super().__init__()
#         CH_LAST = 128
#         NUM_CLASSES = 5
#         # TODO: Change this part to improve your score
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(128, CH_LAST, kernel_size=3, padding=1)
#         # NOTE: input channel should match CH_LAST * (IMAGE_INPUT_SIZE / 2 ** NUM_DOWNSAMPLE) ** 2 
#         self.fc = nn.Linear(CH_LAST *  (INPUT_SIZE // 2 ** 3) ** 2, NUM_CLASSES)
#         self.pool = nn.MaxPool2d(kernel_size=(3, 3), stride=2, padding=1)
        
#     def forward(self, x):
#         h = self.pool(F.relu(self.conv1(x)))
#         h = self.pool(F.relu(self.conv2(h)))
#         h = self.pool(F.relu(self.conv3(h)))
#         return self.fc(h.flatten(1))
    
# model = CassavaClassifier().cuda()
# #model

# Implement the training loop

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,10], gamma=0.1)

In [ ]:
# TODO: Define some hyperparameters here.
# loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:

# TODO: Implement your training loop here.
n_epochs = 15

train_losses = []
val_losses = []
avg_train_losses = []
avg_val_losses = []

best_score = -100
for epoch in range(n_epochs + 1):
    
    print("current learning rate")
    for param_group in optimizer.param_groups:
            print("lr:",param_group['lr'])
    # training loop
    model.train()
    train_correct = 0
    train_total = 0
    
    for batch_idx, samples in enumerate(train_loader):
        x_train, y_train = samples
        
        x_train = x_train.cuda()
        y_train = y_train.cuda()

        y_pred = model(x_train)
        loss = loss_fn(y_pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        train_correct += (torch.max(model(x_train),1)[1] == y_train).sum()
        train_total += x_train.size(0)
        
        if batch_idx % int(10) == 0:
            print('Epoch {:4d}/{} Batch {}/{} Loss: {:.6f}'.format(
                epoch, n_epochs, batch_idx+1, len(train_loader),
                loss.item()
                ))
        
    accuracy = 100 * int(train_correct) / train_total
    print("Train Accuracy ={}".format(accuracy))
    
    # validatoin loop
    model.eval()
    val_correct = 0
    val_total = 0
    
    for batch_idx, samples in enumerate(val_loader):
        x_train, y_train = samples
        
        x_train = x_train.cuda()
        y_train = y_train.cuda()

        y_pred = model(x_train)
        loss = loss_fn(y_pred, y_train)
        
        val_losses.append(loss.item())
        val_correct += (torch.max(model(x_train),1)[1] == y_train).sum()
        val_total += x_train.size(0)
        
    accuracy = 100 *int(val_correct) / val_total
    print("Validation Accuracy ={}".format(accuracy))
    
    
    
    train_loss = np.average(train_losses)
    val_loss = np.average(val_losses)
    avg_train_losses.append(train_loss)
    avg_val_losses.append(val_loss)

    score = -val_loss
    
    if score > best_score:
        best_score = score
        torch.save(model, 'model.pt')
    
    print("best_score:",best_score)    
    
    epoch_len = len(str(n_epochs))

    print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {val_loss:.5f}')

    print(print_msg)
    
    train_losses = []
    val_losses = []
    scheduler.step()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 훈련이 진행되는 과정에 따라 loss를 시각화
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(avg_train_losses)+1),avg_train_losses, label='Training Loss')
plt.plot(range(1,len(avg_val_losses)+1),avg_val_losses,label='Validation Loss')

# validation loss의 최저값 지점을 찾기
#minposs = valid_loss.index(min(valid_loss))+1
#plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
#plt.ylim(0, 0.5) # 일정한 scale
plt.xlim(0, len(avg_train_losses)+1) # 일정한 scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches = 'tight')

In [ ]:
# test loss 및 accuracy을 모니터링하기 위해 list 초기화
test_loss = 0.0
class_correct = list(0. for i in range(5))
class_total = list(0. for i in range(5))

model.eval() # prep model for evaluation
total = 0

for data, target in val_loader :

    data = data.cuda()
    target = target.cuda()
#     if len(target.data) != batch_size:
#         break

    # forward pass: 입력을 모델로 전달하여 예측된 출력 계산
    output = model(data)
    
    # calculate the loss
    loss =loss_fn(output, target)
    # update test loss
    test_loss += loss.item()*data.size(0)
    # 출력된 확률을 예측된 클래스로 변환
    _, pred = torch.max(output, 1)
    # 예측과 실제 라벨과 비교
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # 각 object class에 대해 test accuracy 계산
    
    #for i in range(batch_size):
    for i in range(len(target.data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] +=1
    total += data.size(0)
#     print(total)
#     print(test_loss)
    
# calculate and print avg test loss
test_loss = test_loss/total

print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(5):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %.3f%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 훈련이 진행되는 과정에 따라 loss를 시각화
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(avg_train_losses)+1),avg_train_losses, label='Training Loss')
plt.plot(range(1,len(avg_val_losses)+1),avg_val_losses,label='Validation Loss')

# validation loss의 최저값 지점을 찾기
#minposs = valid_loss.index(min(valid_loss))+1
#plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
#plt.ylim(0, 0.5) # 일정한 scale
plt.xlim(0, len(avg_train_losses)+1) # 일정한 scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches = 'tight')

In [ ]:
# TODO: Implement your training loop here.
# for x, y in data_loader:
#     x, y = x.cuda(), y.cuda()
#     logits = model(x)
#     loss = creterion(logits,)

In [ ]:
def train(model, epochs, train_loader, test_loader, criterion, 
          optimizer, RESULTS_PATH, scheduler=None, MODEL_PATH=None):
    # Run on GPU if available
    device = torch.device("cuda:0")
    print(device)
    model.to(device)
    
    # Training loop
    # -------------------------------
    cols       = ['epoch', 'train_loss', 'train_err', 'test_err']
    results_df = pd.DataFrame(columns=cols).set_index('epoch')
    print('Epoch \tBatch \tNLLLoss_Train')
    
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        model.train()
        running_loss  = 0.0
        best_test_err = 1.0
        for i, data in enumerate(train_loader, 0):   # Do a batch iteration
            
            # get the inputs
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # print average loss for last 50 mini-batches
            running_loss += loss.item()
            if i % 50 == 49:
                print('%d \t%d \t%.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0
        
        if scheduler:
            scheduler.step()
        
        # Record metrics
        model.eval()
        train_loss = loss.item()
        train_err = evaluate(model, train_loader, device)
        test_err = evaluate(model, test_loader, device)
        results_df.loc[epoch] = [train_loss, train_err, test_err]
        results_df.to_csv(RESULTS_PATH)
        print(f'train_err: {train_err} test_err: {test_err}')
        
        # Save best model
        if MODEL_PATH and (test_err < best_test_err):
            torch.save(model.state_dict(), MODEL_PATH)
            best_test_err = test_err
        
        
    
    print('Finished Training')
    model.eval()
    return model

In [ ]:
# Save your trained model. Note that it is recommended to save the 'best' checkpoint, rather than just saving the 'last' checkpoint.
torch.save(model, 'model.pt')

# Verify that your trained model can be loaded

In [ ]:
model = torch.load('/kaggle/working/model.pt')
model.eval()
test_dataset = CassavaDataset('test', transform=TRANSFORMS['test'])
test_loader = DataLoader(test_dataset, batch_size=32)
test_csv = pd.read_csv(TEST_CSV)

y_hats = []
for x, _ in test_loader:
    y_hat = model(x.cuda())
    y_hat = torch.argmax(y_hat,dim=1)
    y_hats.extend(y_hat.cpu().detach().numpy().tolist())
    

test_csv['label'] = y_hats
test_csv[['image_id','label']].to_csv("submission.csv", index=False)
test_csv.head()